## Freelancer SQL Project

I need some SQL queries that will more or less mimic the python code below.

If you're unfamiliar with python the steps are roughly this

1) Download From AWS Athena SQL Server (41k rows) 
2) Isolate Text Column
3) Use Regex to find hashtags
4) Build a column of lists, where each cell contains a list of the hashtags 
5) Input Column of Hashtag Lists into a Multi Label Binarizer/ Dummy Col generator that will output a binary matrix where there is a column for every hashtag and a row for every index/tweet.  There should be a 1 in the rows that contain that hashtag.  
6) Save binary matrix to new SQL table

I have performed all the above steps in Python, but you may have to find work around for the same steps to work in SQL. I'm ultimately concerned about the binary output matrix, and less about your fidelity to the steps above. 

Thank you

Kenan Spruill, Optima Holdings

In [25]:
# import the regex module
import re
import pandas as pd
# function to print all the hashtags in a text
def extract_hashtags(text):
    if type(text) != str:
        return []
    # the regular expression
    regex = "#(\w+)"
     
    # extracting the hashtags
    hashtag_list = re.findall(regex, text)
    return hashtag_list
##you can see how I created this dataset in the cell below which is commented out
df = pd.read_csv("temp.csv")
df.index = df["id"]
#making a smaller dataframe for humans to read the data more easily
ht_df = pd.DataFrame(df["text"], index = df.index)
ht_df["hashtag_list"] =ht_df["text"].apply(extract_hashtags)#getting list of hashtags from text
ht_df["hashtag_count"] = ht_df["hashtag_list"].apply(len) #getting number of hashtags
ht_df = ht_df.sort_values("hashtag_count", ascending = False) #sorting so that hashtags are visible

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer() #initiate binarizer

#create a binarize dataframe from the list of labels
#see documentation: 
#https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html

ht_df_mlb = pd.DataFrame(mlb.fit_transform(ht_df["hashtag_list"]), columns = mlb.classes_, index = ht_df.index)

#again the following commands are just to see the data more clearly and not necesarrily relevant to the project
ht_sum = ht_df_mlb.sum().sort_values(ascending = False)
ht_df_mlb = ht_df_mlb.loc[:,ht_sum.index]
ht_df_mlb

,socialselling,digitalselling,salestips,leadership,sales,digitaltransformation,salesleader,digitalmarketing,salesenablement,marketing,...,drivingtips,drivyn,drone,dyor,eFuels,earnings,earthfromspace,easter,easterkegs,0000FF
id,,,,,,,,,,,,,,,,,,,,,
1547100844717383681,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1543876980092846080,0,0,0,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1551208050601742336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1551208038069149705,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1490390433549238272,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1551654835862339585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1551654198571401218,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1552162208011825152,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#AccessKeys

import json
import pandas as pd
import boto3
import pickle
# importing date class from datetime module
import datetime
import re
import time
import io 
import sklearn
from IPython.display import display, HTML
from pprint import pprint
AWSAccessKeyId = ""
AWSSecretKey = ""
s3 = boto3.client('s3',aws_access_key_id=AWSAccessKeyId,
           aws_secret_access_key= AWSSecretKey)
           
s3x = boto3.resource('s3',aws_access_key_id=AWSAccessKeyId,
           aws_secret_access_key= AWSSecretKey)
           
lambda_client = boto3.client('lambda',aws_access_key_id=AWSAccessKeyId,
           aws_secret_access_key= AWSSecretKey)

BUCKET_NAME = 'optima-audience-data'

bucket_name = "optima-audience-data"

def pull_from_athena(query):
    params = {
        'region': 'us-east-1',
        'database': 'optimaaudience',
        'bucket': 'kfc-athena-output',
        'path': 'temp/athena/output',
        'query': query
    }

    session = boto3.Session(aws_access_key_id  = AWSAccessKeyId, aws_secret_access_key = AWSSecretKey)

    def athena_query(client, params):

        response = client.start_query_execution(
            QueryString=params["query"],
            QueryExecutionContext={
                'Database': params['database']
            },
            ResultConfiguration={
                'OutputLocation': 's3://' + params['bucket'] + '/' + params['path']
            }
        )
        return response

    def athena_to_s3(session, params, max_execution = 60):
        client = session.client('athena', region_name=params["region"])
        execution = athena_query(client, params)
        execution_id = execution['QueryExecutionId']
        state = 'RUNNING'
        i = 1
        while (max_execution > 0 and state in ['RUNNING', 'QUEUED']):
            print("Pull From Athena Attempt: ", i )
            i+=1
            max_execution = max_execution - 1
            response = client.get_query_execution(QueryExecutionId = execution_id)

            if 'QueryExecution' in response and \
                    'Status' in response['QueryExecution'] and \
                    'State' in response['QueryExecution']['Status']:
                state = response['QueryExecution']['Status']['State']
                if state == 'FAILED':
                    print("FAILED")
                    pprint(response)
                    return False
                elif state == 'SUCCEEDED':
                    print("PULL SUCCEEDED:")
                    s3_path = response['QueryExecution']['ResultConfiguration']['OutputLocation']
                    print("LOAD FILE: ", s3_path)
                    filename = re.findall('.*\/(.*)', s3_path)[0]
                    
                    return filename
            time.sleep(3)

        return False

    s3_filename = athena_to_s3(session, params)
    my_path = params['path'] + "/" + s3_filename
    my_path
    key = my_path
    s3_object = s3.get_object(Bucket=params['bucket'], Key=key)
    df = pd.read_csv(io.BytesIO(s3_object['Body'].read()))
    
    return df

def pull_data_from_athena():
    
    QUERY = """
    SELECT 
        * 
    FROM
        (
        SELECT 
                id,
                handle, 
                postdate, 
                favorite_count,
                comment_count,
                text,
                base_acct,
                acct,
                maxfavs,
                ROW_NUMBER() OVER(PARTITION BY id ORDER BY favorite_count DESC) AS rn
            FROM
                (
                SELECT 
                    id,
                    handle, 
                    postdate, 
                    favorite_count,
                    comment_count,
                    text,
                    base_acct,
                    acct 
                FROM

                    "audiencetwittertweets" 
                WHERE 
                    base_acct IN ('elonmusk')
                    )
                    AS table1
                INNER JOIN
                    (SELECT * FROM "audiencetwittermaxfavs") 
                    AS maxtable
                ON 
                table1.id = maxtable.id0
            ) as tmp
    WHERE rn = 1    
            
    """
    print("##########")
    print("ATHENA QUERY:")
    print(QUERY)
    print("##########")

    df = pull_from_athena(QUERY)

    return df

df = pull_data_from_athena()
df.to_csv("temp.csv")